In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, take_log=True, split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_concat_conv_lstm import *

In [10]:
graph_name = 'SpectrogramConcatConvLSTM'  # Specify graph class name

g_cnfg = Config('graph_01')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

g_cnfg.conv1_n_filters = 16
g_cnfg.conv1_kernel_size = 5
g_cnfg.conv1_mp_size = 5
g_cnfg.conv1_mp_strides = 2

g_cnfg.conv2_n_filters = 32
g_cnfg.conv2_kernel_size = 3
g_cnfg.conv2_mp_size = 3
g_cnfg.conv2_mp_strides = 2

g_cnfg.lstm_state_sizes = [100, 200]
g_cnfg.lstm_dropout = {'in': 'y', 'out': 'n', 'state': 'n'}

g_cnfg.X_conv_h = math.ceil(math.ceil(g_cnfg.X_img_h / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
g_cnfg.X_conv_w = math.ceil(math.ceil(g_cnfg.X_img_w / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
g_cnfg.n_conv_flat = g_cnfg.X_conv_h * g_cnfg.X_conv_w * g_cnfg.conv2_n_filters
g_cnfg.n_flat = g_cnfg.n_conv_flat + g_cnfg.lstm_state_sizes[-1]

g_cnfg.n_hidden1 = 2000
g_cnfg.n_hidden2 = 50

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [11]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_01
X_conv_h: 33
X_conv_w: 18
X_img_h: 129
X_img_w: 71
Y_vector_len: 31
conv1_kernel_size: 5
conv1_mp_size: 5
conv1_mp_strides: 2
conv1_n_filters: 16
conv2_kernel_size: 3
conv2_mp_size: 3
conv2_mp_strides: 2
conv2_n_filters: 32
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_dropout: {'state': 'n', 'out': 'n', 'in': 'y'}
lstm_state_sizes: [100, 200]
n_conv_flat: 19,008
n_flat: 19,208
n_hidden1: 2,000
n_hidden2: 50

name: run_01
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 367
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

SpectrogramConcatConvLSTM_graph_01_run_01
Training starts @ 12/11/2017 21:38:38
Epoch 1 Step 1 Best model saved @ 12/11/2017 21:38:45 [Ave valid] 3.437
Epoch 1 Step 1 ends @ 12/11/2017 21:38:45 [Train] 3.432, 5.3% [Valid] 4.1% [Ave valid] 3.437
Epoch 1 Step 50 Best model saved @ 12/11

Epoch 16 Step 5,872 ends @ 12/12/2017 01:19:51 [Train] 0.017, 100.0% [Valid] 90.6% [Ave valid] 0.339
Epoch 17 Step 6,239 ends @ 12/12/2017 01:33:37 [Train] 0.022, 100.0% [Valid] 91.0% [Ave valid] 0.331
INFO:tensorflow:checkpoints/SpectrogramConcatConvLSTM_graph_01_run_01/hourly/model-6400 is not in all_model_checkpoint_paths. Manually adding it.
Epoch 18 Step 6,400 Hourly model saved @ 12/12/2017 01:39:42
Epoch 18 Step 6,606 ends @ 12/12/2017 01:47:27 [Train] 0.008, 100.0% [Valid] 91.6% [Ave valid] 0.349
Epoch 19 Step 6,973 ends @ 12/12/2017 02:01:14 [Train] 0.003, 100.0% [Valid] 91.3% [Ave valid] 0.378
Epoch 20 Step 7,340 ends @ 12/12/2017 02:15:01 [Train] 0.003, 100.0% [Valid] 91.7% [Ave valid] 0.338
Epoch 21 Step 7,707 ends @ 12/12/2017 02:28:51 [Train] 0.005, 100.0% [Valid] 92.4% [Ave valid] 0.330
Epoch 22 Step 7,800 Best model saved @ 12/12/2017 02:32:22 [Ave valid] 0.318
INFO:tensorflow:checkpoints/SpectrogramConcatConvLSTM_graph_01_run_01/hourly/model-8000 is not in all_model_ch

KeyboardInterrupt: 